# Interactive Lecture: Comparison of Fitting Algorithms

In this exercise we will compare the effectiveness of two different fitting algorithms, classic *Least Squares* and a more rigorous *Orthogonal Distance Regression* approach.

| It is structured as follows:                                                             |
| ---------------------------------------------------------------------------------------- |
|1. Define Example Measurement                                                             |
|2. Define Test *True* Dataset                                                             |
|3. Simulate *Measured* Dataset                                                            |
|4. Fit the data with two approaches - *Least Squares* and *Othogonal Distance Regression* |
|5. How do the fitting methods compare?                                                    |
| *Extension Exercises* |

NB: To download the non-interactive, completed version of this notebook follow this [link](https://www.dropbox.com/s/zw8ahlggabjgsxu/harmonisation_interactive_lecture_complete.html?dl=0)

**``First import required python modules``**

To run a Python cell in the iPython notebook select it and press [shift] + [Enter]

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pylab as plt
from scipy.optimize import curve_fit    # scipy least squares implementation
from scipy.odr import *                 # scipy othogonal-distance-regression implementation

---


## 1. Define Example Measurement

Take a simple straight-line measurement function as an example,

$y = a~x + b$

where,
* y is the measurand
* x is it the observed quantity
* a, b are the calibration parameters

In this example set,

* a = 1.0
* b = 0.0

In [ ]:
# Set true calibration parameters
a_true = 1.0
b_true = 0.0

### ``i. Define measurement function in Python``

In [ ]:
def measurement_function(x, a, b):
    """
    Return measurand for straight line measurement function
    
    :parameters:
    x (numpy.ndarray) - measurement observed quantity
    a (float) - calibration parameter
    b (float) - calibration parameter
    
    :return:
    y - measurement measurand
    """
    
    y = a*x + b

    return y

### ``ii. Test our measurement function``


In [ ]:
# Choose test measurement observed quantity
x_test = 2.0

# Compute y from test observed quantity
y_test = measurement_function(x_test, a_true, b_true)

print("Input:")
print("x =", x_test)

print("\nOutput:")
print("y =", y_test)

## 2. Define Test *True* Dataset

Define set of true values for $x$ between $0 \leq x \leq 1$.

### ``i. Define dataset``

In [ ]:
# Number of measurements
n_obs = 100

# x values randomly drawn between 0 and 1 (rounded to 2 decimal places)
x_true = np.round(np.random.rand(n_obs), decimals=2)

# Evaluate y from x values with measurement function
y_true = measurement_function(x_true, a_true, b_true)

### ``ii. Plot dataset``

In [ ]:
# Plot true data
plt.figure()
plt.plot(x_true, y_true, "bo")
plt.ylabel("$y$")
plt.xlabel("$x$")

## 3. Simulate *Measured* Dataset

In reality each has an unknown error measurement,

$y_{data} = y_{true} + \epsilon_{y}$

$x_{data} = x_{true} + \epsilon_{x}$

In our example lets assume each error is independent for each measurement. So for each mesaurement:

* $\epsilon_x \in Norm[\mu=0, \sigma=u_x]$ -- i.e. $\epsilon_{x}$ is a draw from the uncertainty distribution of $x$
* $\epsilon_y \in Norm[\mu=0, \sigma=u_y]$ -- i.e. $\epsilon_{y}$ is a draw from the uncertainty distribution of $y$


Set uncertainty for each measurement to be:

* $u(x) = 0.05$
* $u(y) = 0.05$

In [ ]:
# Define uncertainties
u_x = np.full(n_obs, 0.05)
u_y = np.full(n_obs, 0.05)

# Print uncertainty for first 10 observations
print("u_x:", u_x[:10])
print("u_y:", u_x[:10])

### ``i. Write a function to add errors to our data``

In [ ]:
def add_errors(data, uncertainty):
    """
    Return array with independent random gaussian error each element
    
    :parameters:
    data (numpy.ndarray) - input array
    uncertainty (float) - random error gaussian standard deviation
    
    :return:
    data_error (numpy.ndarray) - input array + independent random gaussian errors
    """
    
    # Determine errors for each true value
    errors = np.random.normal(loc=0, scale=uncertainty, size=data.shape[0])
    
    # Add error to true data to
    data_error = data + errors
    
    return data_error

### ``ii. Simulate set of measured data by adding errors to true values``

In [ ]:
# Add errors to true data to simulate set of "measured data"
x_data = add_errors(x_true, u_x)
y_data = add_errors(y_true, u_y)

### ``iii. Plot measured dataset``

In [ ]:
# Plot results
plt.figure()

# a. True data line
plt.plot(x_true, y_true, 'blue', label="True")

# b. Plot measured data points (with errorbars)
plt.plot(x_data, y_data, "ro", label="Data")

plt.ylabel("$y$")
plt.xlabel("$x$")
plt.legend(loc=4)

## 4. Fitting the Data

In this section we will attempt to fit the measured data we produced in *Section 3* with two difference approaches, standard *Least Squares* (LSQ) or a more rigorous *Orthogonal Distance Regression* (ODR) approach.

**a. Least Squares (LSQ)**

*Least Squares* optimisation fits based on weighting of uncertainties in $y$ dimension only, i.e,

In [ ]:
# Plot LSQ uncertainty picture
plt.figure()

# a. True data line
plt.plot(x_true, y_true, 'blue', label="True")

# b. Plot measured data points (with y errorbars)
plt.errorbar(x=x_data, y=y_data, yerr=u_y, fmt="ro", ecolor='black', label="Data")

plt.ylabel("$y$")
plt.xlabel("$x$")
plt.legend(loc=4)

### ``i. Define function to perform Least Squares fitting.``

In [ ]:
# Use scipy LSQ implementation curve_fit

def fit_lsq(measurement_function, x_data, y_data, u_y, p_initial):
    """
    Return optimised parameters for a given measurement function and input data
    
    :parameters:
    measurement_function (func) - measurement function
    x_data (numpy.ndarray) - input x data
    y_data (numpy.ndarray) - input y data
    u_y (float) - y data uncertainty
    p_initial (list) - intial parameter estimates
    
    :return:
    p_est (list) - optimised parameter estimates
    p_cov (numpy.ndarray) - parameter covariance matrix
    """

    p_est, p_cov = curve_fit(measurement_function, x_data, y_data, p_initial, u_y)

    return p_est, p_cov

### ``ii. Run optimisation``

In [ ]:
# Run optimisation
p_lsq, cov_lsq = fit_lsq(measurement_function, x_data, y_data, u_y, p_initial=[0.0, 0.0])

a_lsq = p_lsq[0]
b_lsq = p_lsq[1]

# Print result
print("LSQ Parameter Estimate:")
print("a =", a_lsq)
print("b =", b_lsq)
print("\nCovariance matrix:")
print(cov_lsq)

### ``iii. Plot results``

In [ ]:
# Plot results
plt.figure()

# a. True calibration line
plt.plot(x_true, y_true, 'blue', label="True")

# b. Plot measured data points
plt.errorbar(x=x_data, y=y_data, yerr=u_y, fmt="ro", ecolor="black", label="Data")

#c. Plot estimate
y_lsq = measurement_function(x_true, a_lsq, b_lsq)
plt.plot(x_true, y_lsq, 'orange', label="LSQ Fit")

plt.ylabel("$y$")
plt.xlabel("$x$")
plt.legend(loc=4)

**b. Orthogonal Distance Regression (ODR)**

*Orthogonal Distance Regression* optimisation methods weight based on uncertainties in *both* the $x$ and $y$ dimension, i.e,

In [ ]:
# Plot ODR uncertainty picture
plt.figure()

# a. True data line
plt.plot(x_true, y_true, 'blue', label="True")

# b. Plot measured data points (with errorbars)
plt.errorbar(x=x_data, xerr=u_x, y=y_data, yerr=u_y, fmt="ro", ecolor='black', label="Data")

plt.ylabel("$y$")
plt.xlabel("$x$")
plt.legend(loc=4)

### ``i. Define function to perform Orthogonal Distance Regression fitting``

In [ ]:
# Use scipy ODRPACK implementation 

def fit_odr(measurement_function, x_data, y_data, u_x, u_y, p_initial):
    """
    Return optimised parameters for a given measurement function and input data
    
    :parameters:
    measurement_function (func) - measurement function
    x_data (numpy.ndarray) - input x data
    y_data (numpy.ndarray) - input y data
    u_y (float) - x data uncertainty
    u_y (float) - y data uncertainty
    p_initial (list) - intial parameter estimates
    
    :return:
    p_est (list) - optimised parameter estimates
    p_cov (numpy.ndarray) - parameter covariance matrix
    """

    odr = ODR(RealData(x=x_data, y=y_data, sx=u_x, sy=u_y), Model(measurement_function), beta0=p_initial)
    result = odr.run()

    p_est = result.beta
    p_cov = result.cov_beta

    return p_est, p_cov

### ``ii. Run optimisation``

In [ ]:
# scipy ODRPACK needs a slightly different form of input to the measurement function
def measurement_function_odr(p, x):
    return p[0] * x + p[1]

# Run optimisation
p_odr, cov_odr = fit_odr(measurement_function_odr, x_data, y_data, u_x, u_y, p_initial=[0.0, 0.0])
a_odr = p_odr[0]
b_odr = p_odr[1]

# Print result
print("ODR Parameter Estimate:")
print("a =", a_odr)
print("b =", b_odr)
print("\nCovariance matrix:")
print(cov_odr)

### ``iii. Plot results``

In [ ]:
# Plot results
plt.figure()

# a. True calibration line
plt.plot(x_true, y_true, 'blue', label="True")

# b. Plot measured data points
plt.errorbar(x=x_data, y=y_data, xerr=u_x, yerr=u_y, fmt="ro", ecolor="black", label="Data")

#c. Plot ODR estimate
y_odr = measurement_function(x_true, a_odr, b_odr)
plt.plot(x_true, y_odr, 'orange', label="ODR Fit")

plt.ylabel("$y$")
plt.xlabel("$x$")
plt.legend(loc=4)

## 5. How well does each perform

Perform Monte Carlo analysis of fitting approaches:
1. Generate ``n_trials`` instances of measured data, each with different random errors
2. Fit each iteration of with both *Least Squares* and *Othogonal Distance Regression* approaches
3. Compare results

### ``i. Perform Monte Carlo analysis``

In [ ]:
# Define number of trials to perform
n_trials = 1000

# Initialise arrays to store parameter results 
a_lsqs = np.zeros(n_trials)
b_lsqs = np.zeros(n_trials)

a_odrs = np.zeros(n_trials)
b_odrs = np.zeros(n_trials)

y_lsqs = np.zeros((n_trials, n_obs))
y_odrs = np.zeros((n_trials, n_obs))

for i in range(n_trials):
    
    # Add errors to true data to generate set of "measured data"
    x_data_i = add_errors(x_true, u_x)
    y_data_i = add_errors(y_true, u_y)
    
    # Least squares fitting    
    p_lsq_i, cov_lsq_i = fit_lsq(measurement_function, x_data_i, y_data_i, u_y, p_initial=[0.0, 0.0])
    a_lsqs[i] = p_lsq_i[0]
    b_lsqs[i] = p_lsq_i[1]
    y_lsqs[i, :] = measurement_function(x_true, a_lsqs[i], b_lsqs[i])
    
    # ODR fitting
    p_odr_i, cov_odr_i = fit_odr(measurement_function_odr, x_data_i, y_data_i, u_x, u_y, p_initial=[0.0, 0.0])
    a_odrs[i] = p_odr_i[0]
    b_odrs[i] = p_odr_i[1]
    y_odrs[i, :] = measurement_function(x_true, a_odrs[i], b_odrs[i])

### ``ii. Plot Outputs``

In [ ]:
# Plot results for a parameter
plt.figure()

# a_lsq estimates
plt.subplot(2, 1, 1)
plt.hist(a_lsqs-a_true, bins=20)
plt.title("a_lsq - a_true")
plt.xlim([-0.1, 0.1])

# a_odr estimates
plt.subplot(2, 1, 2)
plt.hist(a_odrs-a_true, bins=20)
plt.title("a_odr - a_true")
plt.xlim([-0.1, 0.1])

In [ ]:
# Plot results for b parameter
plt.figure()

# b_lsq estimates
plt.subplot(2, 1, 1)
plt.hist(b_lsqs-b_true, bins=20)
plt.title("b_lsq - b_true")
plt.xlim([-0.1, 0.1])

# b_odr estimates
plt.subplot(2, 1, 2)
plt.hist(b_odrs-b_true, bins=20)
plt.title("b_odr - b_true")
plt.xlim([-0.1, 0.1])

In [ ]:
# Plot results of impact on a given y
plt.figure()

# y_lsqs[0] estimates
plt.subplot(2, 1, 1)
plt.hist(y_lsqs[:,0]-y_true[0], bins=20)
plt.title("y_lsq[0] - y_true[0]")
plt.xlim([-0.05, 0.05])

# y_odrs[0] estimates
plt.subplot(2, 1, 2)
plt.hist(y_odrs[:,0]-y_true[0], bins=20)
plt.title("y_odr[0] - y_true[0]")
plt.xlim([-0.05, 0.05])

---

## EXTENSION EXERCISES

The cell below contains the code we used in interactive lecture above. Try editing it to investigate the effects of the following:

1. If the ``x_data`` fitted has no errors.
2. If the ``x_data`` fitted has no errors or uncertainty (use a small number [i.e. 0.0001] - *Orthogonal Distance Regression Methods* cannot accept 0 uncertainty).
3. If the actual uncertainty used to generate errors in the simulated measurements, ``x_data`` and/or ``y_data``, is different to the uncertainty given to the fitting algorithms. Try overestimating and underestimating.
4. If uncertainty values vary between points - i.e. not just 0.05 for every point (could try a function like [np.linspace](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) or [np.random.rand](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html) instead of [np.full](https://docs.scipy.org/doc/numpy/reference/generated/numpy.full.html)).

In [ ]:
# Define uncertainties
u_x = np.full(n_obs, 0.05)
u_y = np.full(n_obs, 0.05)

# Define number of trials to perform
n_trials = 1000

# Initialise arrays to store parameter results 
a_lsqs = np.zeros(n_trials)
b_lsqs = np.zeros(n_trials)

a_odrs = np.zeros(n_trials)
b_odrs = np.zeros(n_trials)

y_lsqs = np.zeros((n_trials, n_obs))
y_odrs = np.zeros((n_trials, n_obs))

for i in range(n_trials):
    
    # Add errors to true data to generate set of "measured data"
    x_data_i = add_errors(x_true, u_x)
    y_data_i = add_errors(y_true, u_y)
    
    # Least squares fitting    
    p_lsq_i, cov_lsq_i = fit_lsq(measurement_function, x_data_i, y_data_i, u_y, p_initial=[0.0, 0.0])
    a_lsqs[i] = p_lsq_i[0]
    b_lsqs[i] = p_lsq_i[1]
    y_lsqs[i, :] = measurement_function(x_true, a_lsqs[i], b_lsqs[i])
    
    # ODR fitting
    p_odr_i, cov_odr_i = fit_odr(measurement_function_odr, x_data_i, y_data_i, u_x, u_y, p_initial=[0.0, 0.0])
    a_odrs[i] = p_odr_i[0]
    b_odrs[i] = p_odr_i[1]
    y_odrs[i, :] = measurement_function(x_true, a_odrs[i], b_odrs[i])
    

# Plot results altogether
plt.figure(figsize=(8, 12))

# a_lsq estimates
plt.subplot(3, 2, 1)
plt.hist(a_lsqs-a_true, bins=20)
plt.title("a_lsq - a_true")
plt.xlim([-0.1, 0.1])

# a_odr estimates
plt.subplot(3, 2, 2)
plt.hist(a_odrs-a_true, bins=20)
plt.title("a_odr - a_true")
plt.xlim([-0.1, 0.1])

# b_lsq estimates
plt.subplot(3, 2, 3)
plt.hist(b_lsqs-b_true, bins=20)
plt.title("b_lsq - b_true")
plt.xlim([-0.1, 0.1])

# b_odr estimates
plt.subplot(3, 2, 4)
plt.hist(b_odrs-b_true, bins=20)
plt.title("b_odr - b_true")
plt.xlim([-0.1, 0.1])

# y_lsqs[0] estimates
plt.subplot(3, 2, 5)
plt.hist(y_lsqs[:,0]-y_true[0], bins=20)
plt.title("y_lsq[0] - y_true[0]")
plt.xlim([-0.05, 0.05])

# y_odrs[0] estimates
plt.subplot(3, 2, 6)
plt.hist(y_odrs[:,0]-y_true[0], bins=20)
plt.title("y_odr[0] - y_true[0]")
plt.xlim([-0.05, 0.05])